In [3]:
import pyreadstat
import pandas as pd
import numpy as np
import networkx as nx
import warnings
import itertools
from tqdm import tqdm
import seaborn as sns
import numpy as np

In [5]:
df, meta = pyreadstat.read_sav("../../data/raw/OD_2017.sav")
df.drop_duplicates(subset=['zona', 'zona_esc', 'zonatra1', 'id_pess'], inplace=True)

In [6]:
df = df[df['muni_dom'] == 36].copy()
max_zona = int(df.zona.max())

In [7]:
df.shape

(57496, 126)

In [8]:
df.head()

,zona,muni_dom,co_dom_x,co_dom_y,id_dom,f_dom,fe_dom,dom,cd_entre,data,...,duracao,modoprin,tipvg,pag_viag,tp_esauto,vl_est,pe_bici,via_bici,tp_estbici,id_ordem
0,1.0,36.0,333743.0,7394463.0,00010001,1.0,15.416667,1.0,1.0,06092017,...,10.0,16.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0
2,1.0,36.0,333743.0,7394463.0,00010001,0.0,15.416667,1.0,1.0,06092017,...,50.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0
5,1.0,36.0,333814.0,7394428.0,00010002,1.0,15.416667,2.0,1.0,06092017,...,10.0,16.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,6.0
9,1.0,36.0,333814.0,7394428.0,00010002,0.0,15.416667,2.0,1.0,06092017,...,10.0,16.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,10.0
13,1.0,36.0,333814.0,7394428.0,00010002,0.0,15.416667,2.0,1.0,06092017,...,20.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,14.0


In [9]:
"""
1 - Agrícola
2 - Construção Civil
3 - Indústria
4 -Comércio
5 - Serviço de Transporte de Carga
6 - Serviço de Transporte de Passageiros
7 - Serviço Creditício-financeiro
8 - Serviço Pessoal
9 - Serviço de Alimentação
10 - Serviço de Saúde
11 - Serviço de Educação
12 - Serviço Especializado
13 - Serviço de Administração Pública
14 - Outros Serviços
"""

job_sector = {
    'essential' : [10],
    'high_impact': [1, 2, 3, 5, 6, 9],
    'regular': [4, 7, 8, 12, 13, 14, 11]
}

In [10]:
def job_criticity(row):
    if row.setor1 in job_sector['essential'] or row.ocup1==10:
        return 4
    if row.setor1 in job_sector['high_impact']:
        return 3
    if row.setor1 in job_sector['regular']:
        return 2
    return np.nan

In [11]:
df['job_level'] = df.apply(job_criticity, axis=1)

In [12]:
"""mask = df['job_level'] == -1
df.loc[mask, 'co_esc_x'] = df.loc[mask, 'co_tr1_x']
df.loc[mask, 'co_esc_y'] = df.loc[mask, 'co_tr1_y']
df.loc[mask, 'zona_esc'] = df.loc[mask, 'zonatra1']
df.loc[mask, 'zonatra1'] = np.nan
df.loc[mask, 'co_tr1_x'] = np.nan
df.loc[mask, 'co_tr1_y'] = np.nan"""

"mask = df['job_level'] == -1\ndf.loc[mask, 'co_esc_x'] = df.loc[mask, 'co_tr1_x']\ndf.loc[mask, 'co_esc_y'] = df.loc[mask, 'co_tr1_y']\ndf.loc[mask, 'zona_esc'] = df.loc[mask, 'zonatra1']\ndf.loc[mask, 'zonatra1'] = np.nan\ndf.loc[mask, 'co_tr1_x'] = np.nan\ndf.loc[mask, 'co_tr1_y'] = np.nan"

In [13]:
trab_esc = df[['zona', 'zona_esc', 'zonatra1', 'id_pess', 'co_tr1_x', 'co_tr1_y', 
               'co_esc_x', 'co_esc_y', 'co_dom_x', 'co_dom_y', 'id_dom', 'idade',
               'criteriobr', 'renda_fa', 'grau_ins', 'job_level', 'estuda']].copy()

trab_esc.rename(columns={'zona':'home', 'co_dom_x' : 'home_x', 'co_dom_y' : 'home_y',
                         'zona_esc':'school', 'co_esc_x' : 'school_x', 
                         'co_esc_y' : 'school_y', 'zonatra1':'work', 'estuda': 'studies',
                         'co_tr1_x': 'work_x', 'co_tr1_y': 'work_y', 'id_pess': 'id',
                         'id_dom': 'home_id', 'grau_ins': 'education'}, inplace=True)

trab_esc = trab_esc[~(trab_esc['work'] > max_zona) & 
        ~(trab_esc['school'] > max_zona) & ~(trab_esc['home'] > max_zona)].reset_index(drop=True)

In [14]:
# Beneficiarios do Sistema de Saude Privado

#https://portal.cfm.org.br/images/PDF/leitosdeuticapitais2018.pdf
quantile = 1 - 5730524/12038175
print('Tem plano de saude aqueles com renda familiar superior a:')
print(np.quantile(trab_esc['renda_fa'], quantile))

Tem plano de saude aqueles com renda familiar superior a:
4000.0


In [15]:
def has_private_healthcare(family_income):
    return family_income >= 4000

trab_esc['private_healthcare'] = trab_esc['renda_fa'].apply(has_private_healthcare)

In [18]:
trab_esc.to_feather("../../data/interim/work_school_home_sp_esc.feather")